In [1]:
import os
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [3]:
load_dotenv()

True

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [6]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small"
)

In [7]:
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

In [8]:
collection = chroma_client.get_or_create_collection('blog', embedding_function=openai_ef)

In [9]:
vectorstore = Chroma(client=chroma_client, collection_name='blog', embedding_function=OpenAIEmbeddings())

In [10]:
retriever = vectorstore.as_retriever()

#### Crear una Chain personalizada

In [18]:
system_prompt = (
    'Usted es un asistente para tareas de respuesta a preguntas. '
    'Si el contexto de entrada esta en un idioma diferente al español, traduzca el texto al español.'
    'Utilice las siguientes piezas de contexto recuperado para responder '
    'la pregunta. Si no sabes la respuesta, di que No lo sé.'
    'Utilice tres oraciones como máximo y mantenga larespuesta concisa.'
    'SIMPRE RESPONDA EN ESPAÑOL.'
    '\n\n'
    '{context}'
)

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [22]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain_2 = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain_2.invoke({"input": "What is Influence Functions? get me the examples of Influence Functions"})
print(response["answer"])

Las funciones de influencia son una herramienta matemática que se utiliza para medir el impacto de un punto de datos individual en el resultado de un modelo de aprendizaje automático. En otras palabras, nos dicen qué tan importante es un punto de datos específico para el modelo.

Un ejemplo de función de influencia es la función de influencia de eliminación, que mide el cambio en el modelo cuando se elimina un punto de datos específico. Otra función de influencia es la función de influencia de ponderación, que mide el cambio en el modelo cuando se cambia el peso de un punto de datos específico.

Las funciones de influencia se pueden utilizar para identificar puntos de datos influyentes, que son puntos de datos que tienen un gran impacto en el modelo. Estos puntos de datos pueden ser outliers o puntos de datos que son muy representativos de la distribución de datos.

